## 1 Fetching Data from Supabase

In [ ]:
from supabase import create_client, Client
import pandas as pd
import os as os
from dotenv import load_dotenv

load_dotenv()

# Initialize Supabase client
url: str = os.environ.get("SUPABASE_PUBLIC_URL")
key: str = os.environ.get("SUPABASE_SERVICE_ROLE_KEY")
supabase: Client = create_client(url, key)

# Fetch data from each table
_acceptance_data = supabase.table("_acceptance").select("*").execute().data
_actions_data = supabase.table("_actions").select("*").execute().data
_app_names_data = supabase.table("_app_names").select("*").execute().data
_location_data = supabase.table("_location").select("*").execute().data
_sex = supabase.table("_sex").select("*").execute().data
_weekdays = supabase.table("_weekdays").select("*").execute().data

user_app_usage_data = supabase.table("user_app_usage").select("*").execute().data
users_data = supabase.table("users").select("*").execute().data

In [ ]:
# Convert to pandas DataFrames
df__acceptance = pd.DataFrame(_acceptance_data)
df__actions = pd.DataFrame(_actions_data)
df__app_names = pd.DataFrame(_app_names_data)
df__location = pd.DataFrame(_location_data)
df__sex = pd.DataFrame(_sex)
df__weekdays = pd.DataFrame(_weekdays)

df_user_app_usage = pd.DataFrame(user_app_usage_data)
df_users = pd.DataFrame(users_data)

# Verify the structure of the dataframes
df_user_app_usage.head()

##  2 Data Preprocessing

## 2.1 Remove uncompleted rows/entrys

In [ ]:
def remove_none_rows(df, column_name):
    """
    Removes rows from a DataFrame where the specified column has 'None' or 'NaN'.
    """
    return df.dropna(subset=[column_name])

df_user_app_usage = remove_none_rows(df_user_app_usage, 'app_usage_time')

# Verify the structure of the dataframes
display(df_user_app_usage.head())
display(df_user_app_usage.dtypes)

## 2.2 Calculate/simplify data functions

In [ ]:
from datetime import datetime

# Start functions

def convert_boolean_to_numeric(df_original, column_name):
    """
    Converts a boolean column in a DataFrame to 0 or 1.
    """
    df = df_original.copy()

    # Convert boolean to int (True to 1, False to 0)
    df[column_name] = df[column_name].astype(int)

    return df

def convert_string_to_date(df_original, dob_column):
    """
    Converts a date of birth column from string to datetime and calculates the age.
    """
    df = df_original.copy()
    df[dob_column] = pd.to_datetime(df[dob_column])
    df['age'] = df[dob_column].apply(
        lambda dob: datetime.now().year - dob.year - ((datetime.now().month, datetime.now().day) < (dob.month, dob.day))
    )
    return df

# Final functions

def normalize_numerical_data(df_original, column, fixed_max):
    """
    Normalizes a specified column of the DataFrame.
    """
    df = df_original.copy()

    # Normalize the column
    df[column] = df[column] / fixed_max
    
    # Ensure that the values do not exceed 1, more than 1 are clipped to 1
    df[column] = df[column].clip(lower=0, upper=1)
    
    return df

def one_hot_encoding(df_original: pd.DataFrame, map_df: pd.DataFrame, column_to_encode: str, map_column: str, map_values: str):
    """
    Maps a column to new values and applies one-hot encoding, ensuring all categories are represented.
    """
    df = df_original.copy()
    
    # Map the column to new valuesthan
    mapping = dict(zip(map_df[map_column], map_df[map_values]))
    df[column_to_encode] = df[column_to_encode].map(mapping)

    # One-hot encoding
    df = pd.get_dummies(df, columns=[column_to_encode], prefix=column_to_encode)

    # Add missing columns (if any) and fill with 0
    required_columns = map_df[map_values].unique()
    for col in required_columns:
        full_col_name = f'{column_to_encode}_{col}'
        if full_col_name not in df.columns:
            df[full_col_name] = 0
        df[full_col_name] = df[full_col_name].astype(int)

    return df

def normalize_time(df_original, time_column):
    """
    Normalizes time values in a DataFrame column.
    """
    df = df_original.copy()
    
    # Convert the time column to pandas datetime
    df[time_column] = pd.to_datetime(df[time_column])

    # Normalize time: hour + minute/60 + second/3600, then divide by 24
    df[time_column] = df[time_column].apply(lambda x: (x.hour + x.minute / 60 + x.second / 3600) / 24)

    return df

import hashlib

def hash_encode(df_original: pd.DataFrame, column: str, num_buckets: int):
    """
    Hash-encodes a column in a DataFrame.
    """
    df = df_original.copy()
    
    def hash_column(data):
        return int(hashlib.sha256(str(data).encode()).hexdigest(), 16) % num_buckets

    df[column] = df[column].apply(hash_column)

    return df

### 2.2 Normalize and numericalize data

In [ ]:
# Apply

# df_user_app_usage
df_user_app_usage_normalized = df_user_app_usage.drop(columns=['id', 'created_at'])
df_user_app_usage_normalized = one_hot_encoding(df_user_app_usage_normalized, df__weekdays, 'weekday', 'id', 'weekday')
df_user_app_usage_normalized = normalize_time(df_user_app_usage_normalized, 'time_of_day')
df_user_app_usage_normalized = normalize_numerical_data(df_user_app_usage_normalized, 'app_usage_time', fixed_max=86400) # 24h max
df_user_app_usage_normalized = hash_encode(df_user_app_usage_normalized, 'app_name', 1000) # hash-encoding upto 1000 apps
df_user_app_usage_normalized = one_hot_encoding(df_user_app_usage_normalized, df__acceptance, 'acceptance', 'id', 'acceptance')
df_user_app_usage_normalized = one_hot_encoding(df_user_app_usage_normalized, df__actions, 'action', 'id', 'action')
df_user_app_usage_normalized = one_hot_encoding(df_user_app_usage_normalized, df__location, 'location', 'id', 'location')
df_user_app_usage_normalized = convert_boolean_to_numeric(df_user_app_usage_normalized, 'should_be_blocked')

# df_users
df_users_normalized = convert_string_to_date(df_users, 'date_of_birth')
df_users_normalized = df_users_normalized.drop(columns=['created_at', 'date_of_birth', 'first_name', 'last_name'])
df_users_normalized = normalize_numerical_data(df_users_normalized, 'age', fixed_max=130) # max-age fixed to 130 years
df_users_normalized = one_hot_encoding(df_users_normalized, df__sex, 'sex', 'id', 'sex')


# num_acceptance_categories = df__acceptance['id'].nunique()

# display(num_acceptance_categories)

# Check the results
display(df_user_app_usage_normalized.head())
# display(df_user_app_usage_normalized.dtypes)

display(df_users_normalized.head())
# display(df_users_normalized.dtypes)


Merge data

In [ ]:
def reduce_dataframe_types(df_original):
    """
    Reduces dataframe types.
    """
    df = df_original.copy()
    
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            if col in ["app_name", "user_id"]:
                df[col] = df[col].astype('uint16') # increase this if hash-encoding with larger bucket sizes
            else:
                df[col] = df[col].astype('bool') # all one-hot encoded things
    return df

In [ ]:
merged_df =  df_users_normalized.merge(df_user_app_usage_normalized, left_on='id', right_on='user_id')
merged_df = hash_encode(merged_df, 'user_id', 1000) # hash-encoding upto 1000 users

merged_df = merged_df.drop(columns=['id'])
merged_df = reduce_dataframe_types(merged_df)

display(merged_df.head())
display(merged_df.dtypes)


## 3 TensorFlow Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate
from tensorflow.keras.models import Model

def build_and_compile_model(num_buckets, embedding_dim, feature_columns):
    """
    Builds and compiles a TensorFlow model for the given feature columns.
    """
    # Input Layers
    inputs = {col: Input(shape=(1,), name=col) for col in feature_columns}

    # Embedding for hash-encoded columns
    embeddings = []
    for col in ['user_id', 'app_name']:
        emb = Embedding(input_dim=num_buckets, output_dim=embedding_dim, input_length=1)(inputs[col])
        emb = Flatten()(emb)
        embeddings.append(emb)

    # Directly use other columns
    other_cols = [inputs[col] for col in feature_columns if col not in ['user_id', 'app_name']]
    concatenated_features = Concatenate()(embeddings + other_cols)

    # For simplicity, using Dense layers instead of Transformer
    x = Dense(128, activation='relu')(concatenated_features)
    x = Dense(64, activation='relu')(x)

    # Output Layer for Percentage (regression)
    output = Dense(1, activation='linear')(x)  # 'linear' can be omitted as it is the default

    # Model
    model = Model(inputs=list(inputs.values()), outputs=output)

    # Compile the model for regression
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])  # mae is mean absolute error

    return model

# merged_df is the DataFrame
feature_columns = merged_df.columns.tolist()
# Exclude 'should_be_blocked' from feature columns
feature_columns = [col for col in merged_df.columns if col != 'should_be_blocked']
display(feature_columns)

model = build_and_compile_model(1000, 64, feature_columns)
model.summary()

## 4. Making Predictions

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data
train, test = train_test_split(merged_df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
print("-------------------------------------")

# Prepare the data for the model, label_column is the column that we are trying to predict
def prepare_data(df, feature_columns, label_column):
    features = {col: df[col].values for col in feature_columns if col != label_column}
    labels = df[label_column].values
    
    return features, labels


x_train, y_train = prepare_data(train, feature_columns, 'should_be_blocked')
x_val, y_val = prepare_data(val, feature_columns, 'should_be_blocked')
x_test, y_test = prepare_data(test, feature_columns, 'should_be_blocked')

# print("Train")
# display(x_train)
# display(y_train)

# print("Val")
# display(x_val)
# display(y_val)

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print("-------------------------------------")
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')
print("-------------------------------------")

# Predicting new data
predictions = model.predict(x_test)

# Since the output is now continuous, adjust how you interpret the predictions
# For instance, you might round them to the nearest whole number or percentage
predicted_values = predictions.flatten()  # If predictions need to be flattened
display(predicted_values)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

# Plot training & validation mean absolute error values
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')  # Change 'val_mae' according to what's in your history object
plt.title('Model Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()